# Overview
More or less a clone of `bold_register_scratch.ipynb` only now we are going to rely on manually alignment instead of the always failing ML models from ANTs. I don't mean to entirely imply that it is ANTs' fault, I might be misusing the models. However, it's proving difficult, and iterating over something that takes so long to align is taking forever. So lets go with the manual alignment.

**steps**
We will load all the data, run motion correction, and then manually align

# Setup and Load Data

In [2]:
from notebook_viewer_functions import *
from functions import *
from scivol import *
import numpy as np
import json
import ants
import gzip
import matplotlib.pyplot as plt
from ipywidgets import interact, Button, Output
import pickle
import napari
import sparse
import matplotlib.transforms as mtransforms

proj_root = parent_directory()
print(f"project root: {proj_root}")
t1_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
bold_stim_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-emotionalfaces_run-1_bold.nii.gz")
bold_rest_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-rest_bold.nii.gz")
mni_anat_filepath =  os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_mask_filepath = os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")
events_tsv_path = os.path.join(proj_root, "media/sub-01/func/task-emotionalfaces_run-1_events.tsv")
stimulus_image_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/POFA/fMRI_POFA"
log_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/data/01-subject_emofaces1_2019_Aug_14_1903.log"
cache_folder = "/Volumes/GlyphA_R1/nvol_cache"

raw_t1_img = ants.image_read(t1_input_filepath)
raw_stim_bold = ants.image_read(bold_stim_filepath)
raw_rest_bold_img = ants.image_read(bold_rest_filepath)
mni_img = ants.image_read(mni_anat_filepath)
mni_mask_img = ants.image_read(mni_mask_filepath)

project root: /Users/joachimpfefferkorn/repos/neurovolume


In [3]:
bold_image = ants.image_read(bold_stim_filepath)
t1_image = ants.image_read(t1_input_filepath)

# Functions

In [ ]:
#from GPT
#uses euler angles
translation = np.array([
    [1, 0, 0, tx],
    [0, 1, 0, ty],
    [0, 0, 1, tz],
    [0, 0, 0, 1]
])

scale = np.array([
        [sx, 0,  0,  0],
        [0,  sy, 0,  0],
        [0,  0,  sz, 0],
        [0,  0,  0,  1]
    ])

rotation_x = np.array([
    [1, 0, 0, 0],
    [0, np.cos(rx), -np.sin(rx), 0],
    [0, np.sin(rx), np.cos(rx), 0],
    [0, 0, 0, 1]
])

rotation_y = np.array([
    [cos_ry, 0, sin_ry],
    [0, 1, 0],
    [-sin_ry, 0, cos_ry]
])

rotation_z = np.array([
    [cos_rz, -sin_rz, 0],
    [sin_rz, cos_rz, 0],
    [0, 0, 1]
])

[[1 0 0 5]
 [0 1 0 6]
 [0 0 1 7]
 [0 0 0 1]]


In [5]:
def transform_BOLD(bold_frame: np.ndarray, translation: tuple, scale: tuple, rotation: tuple,):
    tx, ty, tz = translation
    sx, sy, sz = scale
    rx, ry, rz = rotation
    translation = np.array([
        [1, 0, 0, tx],
        [0, 1, 0, ty],
        [0, 0, 1, tz],
        [0, 0, 0, 1]
    ])

# Translation matrices copypasta from GPT
    scale = np.array([
            [sx, 0,  0,  0],
            [0,  sy, 0,  0],
            [0,  0,  sz, 0],
            [0,  0,  0,  1]
        ])

# I wonder where the origin is here...
    rotation_x = np.array([
        [1, 0, 0, 0],
        [0, np.cos(rx), -np.sin(rx), 0],
        [0, np.sin(rx), np.cos(rx), 0],
        [0, 0, 0, 1]
    ])
    rotation_y = np.array([
        [np.cos(ry), 0, np.sin(ry), 0],
        [0, 1, 0, 0],
        [-np.sin(ry), 0, np.cos(ry), 0],
        [0, 0, 0, 1]
    ])
    rotation_z = np.array([
        [np.cos(rz), -np.sin(rz), 0, 0],
        [np.sin(rz), np.cos(rz), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    rotation_matrix = rotation_x @ rotation_y @ rotation_z
    transformation_matrix = translation @ rotation @ scale

In [ ]:
#TODO try torch sparse?



def manual_bold_alignment(bold_seq_vol: np.ndarray, anat_vol: np.ndarray,
                                  dim='x', bold_cmap = 'viridis', anat_cmap = 'gray'):
    def x_coord(slice_idx, frame_idx, opacity, rotation):
        fig, axes = plt.subplots(1,3, figsize=(15,5))

        fig.suptitle('x axis view')

        bold_slice = bold_seq_vol[slice_idx,:,:, frame_idx]

        bold_im = axes[0].imshow(bold_slice, cmap=bold_cmap)
        bold_im.set_transform(mtransforms.Affine2D().rotate_deg(rotation))
        axes[0].set_title('BOLD')

        axes[1].imshow(anat_vol[slice_idx,:,:], cmap=anat_cmap)
        axes[1].set_title('Anatomy')

        axes[2].imshow(anat_vol[slice_idx,:,:], cmap=anat_cmap)
        axes[2].imshow(bold_slice, cmap=bold_cmap, alpha=opacity)
        axes[2].set_title('Overlay')

        return rotation
    
    
    def y_coord(slice_idx, frame_idx, opacity):
        fig, axes = plt.subplots(1,3, figsize=(15,5))

        fig.suptitle('y axis view')

        axes[0].imshow(bold_seq_vol[:,slice_idx,:, frame_idx], cmap=bold_cmap)
        axes[0].set_title('BOLD')

        axes[1].imshow(anat_vol[:,slice_idx,:], cmap=anat_cmap)
        axes[1].set_title('Anatomy')

        axes[2].imshow(anat_vol[:,slice_idx,:], cmap=anat_cmap)
        axes[2].imshow(bold_seq_vol[:,slice_idx,:, frame_idx], cmap=bold_cmap, alpha=opacity)
        axes[2].set_title('Overlay')

    def z_coord(slice_idx, frame_idx, opacity):
        fig, axes = plt.subplots(1,3, figsize=(15,5))

        fig.suptitle('z axis view')

        axes[0].imshow(bold_seq_vol[:,:,slice_idx, frame_idx], cmap=bold_cmap)
        axes[0].set_title('BOLD')

        axes[1].imshow(anat_vol[:,:,slice_idx], cmap=anat_cmap)
        axes[1].set_title('Anatomy')

        axes[2].imshow(anat_vol[:,:,slice_idx], cmap=anat_cmap)
        axes[2].imshow(bold_seq_vol[:,:,slice_idx, frame_idx], cmap=bold_cmap, alpha=opacity)
        axes[2].set_title('Overlay')

    match dim:
        case "x":
            interact(x_coord, slice_idx=(0, anat_vol.shape[0]-1), frame_idx=(0, bold_seq_vol.shape[3]-1),opacity=(0, 1.0), rotation=(0,365))
        case 'y':
            interact(y_coord, slice_idx=(0, anat_vol.shape[1]-1), frame_idx=(0, bold_seq_vol.shape[3]-1),opacity=(0, 1.0))
        case 'z':
            interact(z_coord, slice_idx=(0, anat_vol.shape[2]-1), frame_idx=(0, bold_seq_vol.shape[3]-1),opacity=(0, 1.0))


# The Meat of it All

In [6]:
# Truncated Version
sliced = bold_image.numpy()[:, :, :, :10]
bold_truncated_img = ants.from_numpy(sliced, spacing=bold_image.spacing, origin=bold_image.origin, direction=bold_image.direction)
#stabilized = ants.motion_correction(bold_image)
stabilized_truncated = ants.motion_correction(bold_truncated_img)

In [ ]:
print(bold_truncated_img.numpy().shape)
print(bold_truncated_img.numpy().dtype)
print(t1_image.numpy().dtype)
print(t1_image.numpy().shape[0])

In [8]:
def align_stabilized_bold_to_anat_p1(bold_img, t1_img, template_frame_idx=0, cache_dir=cache_folder):
    """"
    This function aligns a 4D BOLD image to a T1 anatomy image
    by aligning the mean of the BOLD to the T1 anatomy.
    It uses frame registration from only one "template frame"
    as we are assuming you're using a motion corrected
    BOLD image (or something relatively stable) as your input
    
    It gets things in the ballpark, but does will require some
    extra manual registration afterwards.
    """

    #This should also work with a mean image, I wonder what's better? TODO figure out this question
    print("Creating template frame")
    template_frame_idx = ants.from_numpy(bold_img.numpy()[:,:,:,template_frame_idx], spacing=bold_img.spacing[:3])
    frame_registration = ants.registration(
        fixed=t1_img,
        moving=template_frame_idx,
        type_of_transform="Rigid", 
    )
    output_shape = (t1_image.numpy().shape[0], t1_image.numpy().shape[1], t1_image.numpy().shape[2], bold_img.numpy().shape[3])
    registered_frames = np.zeros(shape=output_shape, dtype=bold_image.dtype)

    for frame in range(bold_img.shape[3]):
        print(f"aligning frame {frame + 1}/{bold_img.shape[3]} ")
        bold_frame = ants.from_numpy(bold_img.numpy()[:,:,:,frame],
                                    spacing=bold_img.spacing[:3])
        registered_frame_data = sparse.COO.from_numpy(ants.apply_transforms(
            fixed=t1_img,
            moving=bold_frame,
            transformlist=frame_registration['fwdtransforms'],
            interpolator='linear'
        ).numpy())
        registered_frames[:,:,:,frame] = registered_frame_data
    #sparse_registered_frames = sparse.COO.from_numpy(registered_frames) #Honestly we might not even need sparse and it crashed right at this moment
    #Lets just use the numpy for now and assess the memory load
    print("done")
    return registered_frames

#breaking this up into multiple functions for testing

Registering just the truncated bold back to ANTS was taking upwards of 20 minutes. Upon reflection, I began to wonder if it was even necessary? We're casting this to numpy arrays anyways, why go back to ANTS?

In [ ]:
registered_BOLD = align_stabilized_bold_to_anat_p1(stabilized_truncated['motion_corrected'], t1_image)

Creating template frame
aligning frame 1/10 
aligning frame 2/10 
aligning frame 3/10 
aligning frame 4/10 
aligning frame 5/10 
aligning frame 6/10 
aligning frame 7/10 
aligning frame 8/10 
aligning frame 9/10 
aligning frame 10/10 
done


In [ ]:
manual_bold_alignment(registered_BOLD, t1_image.numpy())

Napari can't smoothly handle a truncated `(10, 512, 512, 296)` registered BOLD image, let alone an entire `185` sequence.

However, it is proving to be an incredibly valuable tool in checking anatomical alignment.

I believe that if I can transform our BOLD matrices in Napari the best workflow is the following:

**Actually let's just do the alignment in matplot lib and then verify it with Napari**

- Skull Strip T1
- Skull Strip Bold (if possible/applicable)
- Register skull stripped T1 and Bold
- Manually align based off a template frame (either the generated mean or a chosen frame) with Napari
- Apply alignment to non-skull stripped versions
- Add all four grids -skull stripped T1, Full T1, Skull stripped BOLD, Full Bold- to `.nervol`